In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.cuda.amp import autocast, GradScaler
import numpy as np

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load and preprocess data
x_train, y_train = np.load("./Karlo/extra/trainset_p2000n10000_train.npz").values()
x_test, y_test = np.load("./Karlo/extra/trainset_p2000n2500_test.npz").values()

x_train[:, :, -1] *= 1e-10
y_train = np.log(y_train[:, -1])
x_test[:, :, -1] *= 1e-10
y_test = np.log(y_test[:, -1])

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(-1)

# Dataset and DataLoader
dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)

# Activation helper
def get_activation(name):
    if name == "leaky_relu":
        return nn.LeakyReLU()
    elif name == "relu":
        return nn.ReLU()
    elif name == "sigmoid":
        return nn.Sigmoid()
    else:
        raise ValueError(f"Unsupported activation: {name}")

In [2]:
# RNN model
class LIVRNN(nn.Module):
    def __init__(self, architecture, input_shape):
        super(LIVRNN, self).__init__()
        self.grus = nn.ModuleList()
        self.norms = nn.ModuleList()
        input_dim = input_shape[-1]

        for i in range(len(architecture["units"]) - 1):
            return_seq = i != (len(architecture["units"]) - 2)
            self.grus.append(nn.GRU(
                input_size=input_dim,
                hidden_size=architecture["units"][i],
                batch_first=True))
            self.norms.append(nn.BatchNorm1d(2000, architecture["units"][i]))
            input_dim = architecture["units"][i]

        self.output_layers = nn.ModuleList([
            nn.Linear(architecture["units"][-2], 1)
            for _ in range(architecture["units"][-1])
        ])
        self.activation = get_activation(architecture["activations"][-1])

    def forward(self, x):
        for gru, norm in zip(self.grus, self.norms):
            x, _ = gru(x)
            x = norm(x)
        outputs = [self.activation(layer(x[:, -1])) for layer in self.output_layers]
        return torch.stack(outputs, dim=1).squeeze(-1)  # shape: [B, N]

In [6]:
# Architecture definition
architecture = {
    "units": [128, 64, 32, 1],
    "activations": ["leaky_relu", "leaky_relu", "leaky_relu", "relu"]
}

model = LIVRNN(architecture, x_train.shape).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [7]:
# Training loop
num_epochs = 400
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)

    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            outputs = model(xb)
            loss = criterion(outputs, yb)
            val_loss += loss.item() * xb.size(0)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss/len(train_dataset):.4f}, "
          f"Val Loss: {val_loss/len(val_dataset):.4f}")

Epoch 1/400 - Train Loss: 1974.4140, Val Loss: 1196.7561
Epoch 2/400 - Train Loss: 370.2803, Val Loss: 47.8958
Epoch 3/400 - Train Loss: 45.0813, Val Loss: 46.2444
Epoch 4/400 - Train Loss: 44.3075, Val Loss: 46.2530
Epoch 5/400 - Train Loss: 44.2876, Val Loss: 46.2975
Epoch 6/400 - Train Loss: 44.3184, Val Loss: 46.2505
Epoch 7/400 - Train Loss: 44.3049, Val Loss: 46.2571
Epoch 8/400 - Train Loss: 44.2961, Val Loss: 46.3089
Epoch 9/400 - Train Loss: 44.3066, Val Loss: 46.3165
Epoch 10/400 - Train Loss: 44.3457, Val Loss: 46.2438
Epoch 11/400 - Train Loss: 44.3161, Val Loss: 46.2854
Epoch 12/400 - Train Loss: 44.3274, Val Loss: 46.2468
Epoch 13/400 - Train Loss: 44.3319, Val Loss: 46.2753
Epoch 14/400 - Train Loss: 44.3075, Val Loss: 46.2557
Epoch 15/400 - Train Loss: 44.3319, Val Loss: 46.2699
Epoch 16/400 - Train Loss: 44.3705, Val Loss: 46.2804
Epoch 17/400 - Train Loss: 44.3035, Val Loss: 46.3254


KeyboardInterrupt: 